# Forecasting time-series on a map
## Yellow Taxi in New York

The project's goal is to learn how to predict the number of trips in the coming hours in each area of New York. To accomplish this, we will ingest the raw data and then aggregate it by hour and region. For simplicity, we will use [Discrete Global Grid H3](https://www.uber.com/en-DE/blog/h3/). The result will be an hourly time-series, each representing the count of trips originating from distinct areas. Before running prediction and visualizing results, we will enrich data with third-party signals, such as information about holidays and offline sports events.

During the project, you will learn how to:
- Work with geospatial data
- Predict time-series of complex structure
- Enrich data with new features

This approach is not unique to trip forecasting but is equally applicable in various scenarios where predictive analysis is required. Examples include forecasting scooter or bike pickups, food delivery orders, sales across multiple retail outlets, or predicting the volume of cash withdrawals across an ATM network. Such models are invaluable for planning and optimization across various industries and services.

## Step 1. Data ingestion

The New York Taxi and Limousine Commission (TLC) has provided detailed, anonymized customer travel data since 2009. Painted yellow cars can pick up passengers in any of the city's five boroughs. Raw data on yellow taxi rides can be found on the [TLC website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). This data is divided into files by month. Each file contains detailed trip information, you can read about it [here](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf). For our project, we will use a compact version of that dataset, it will include the following fields:
- Pickup Time
- Dropoff Time
- Distance
- Pickup Location
- Dropoff Location
- Total Amount

To complete the project, you will need to download and process more than 20GB of raw data. We will simplify this task by putting the dataset on an S3 bucket, which you will connect as an external stage.

In [ ]:
-- Create a database that we will use for this project
create database if not exists aa_quickstart;

In [ ]:
-- Select the newly created database for further use
use aa_quickstart.public;

In [ ]:
-- Create a file format which corresponds to the format of the trip data we stored in S3

CREATE OR REPLACE FILE FORMAT csv_format TYPE = csv
FIELD_OPTIONALLY_ENCLOSED_BY = '"' FIELD_DELIMITER = ';' compression='gzip';

In [ ]:
-- We need this step because the Marketplace currently lacks PredictHQ events data for 2014-2015. We will remove it later.

CREATE OR REPLACE FILE FORMAT csv_format_nocompression TYPE = csv
FIELD_OPTIONALLY_ENCLOSED_BY = '"' FIELD_DELIMITER = ',';

In [ ]:
-- Create an external stage using S3 with test data

CREATE OR REPLACE STAGE aa_stage URL = 's3://sfquickstarts-obielov/advanced_analytics/';

In [ ]:
-- Create a table where we will store the New York taxi pickups dataset

CREATE OR REPLACE TABLE ny_taxi_rides (vendor_id STRING, 
                                 pickup_datetime TIMESTAMP,
                                 dropoff_datetime TIMESTAMP,
                                 distance FLOAT,
                                 pickup_location GEOGRAPHY,
                                 dropoff_location GEOGRAPHY,
                                 total_amount FLOAT);

In [ ]:
-- Fill ny_taxi_rides table with test data: 19Min on Medium

COPY INTO ny_taxi_rides
FROM @aa_stage/yellow_taxi
FILE_FORMAT = (FORMAT_NAME = csv_format);

In [ ]:
-- Creating table for PredictHQ data
-- We need this step because the Marketplace currently lacks PredictHQ events data for 2014-2015. We will remove it later.

create or replace TABLE aa_quickstart.PUBLIC.PREDICTHQ_DATA (
	TITLE VARCHAR(16777216),
	LABELS VARCHAR(16777216),
	PHQ_LABELS VARCHAR(16777216),
	CATEGORY VARCHAR(16777216),
	DESCRIPTION VARCHAR(16777216),
	START_TIME TIMESTAMP_NTZ(9),
	END_TIME TIMESTAMP_NTZ(9),
	GEOG GEOGRAPHY,
	VENUE_ID VARCHAR(16777216),
	VENUE_NAME VARCHAR(16777216),
	VENUE_FORMATTED_ADDRESS VARCHAR(16777216),
	SCOPE VARCHAR(16777216),
	RANK NUMBER(38,0),
	PRIVATE BOOLEAN
);

In [ ]:
-- Loading PredictHQ data
-- We need this step because the Marketplace currently lacks PredictHQ events data for 2014-2015. We will remove it later.

COPY INTO PREDICTHQ_DATA
FROM @aa_stage/predicthq.csv
FILE_FORMAT = (FORMAT_NAME = csv_format_nocompression);

## Step 2. Data transformation


In this step, we'll divide New York into uniformly sized regions and assign each taxi pick-up location to one of these regions. Our goal is to get a table with the number of taxi trips per hour for each region.

To achieve this division, we will use the Discrete Global Grid H3. H3 organizes the world into a grid of equal-sized hexagonal cells, with each cell identified by a unique code (either a string or an integer). This hierarchical grid system allows cells to be combined into larger cells or subdivided into smaller ones, facilitating efficient geospatial data processing.

H3 offers 16 different resolutions for dividing areas into hexagons, ranging from resolution 0, where the world is segmented into 122 large hexagons, to resolution 15. At this resolution, each hexagon is less than a square meter, covering the world with approximately 600 trillion hexagons. You can read more about resolutions [here](https://h3geo.org/docs/core-library/restable/). For our task, we will use resolution 8, where the size of each hexagon is about 0.7 sq. km (0.3 sq. miles).

Since on resolution 8, we might have more than 1000 hexagons for New York, to speed up the training process you will keep only hexagons where in 2014 were more than 1M pickups. It's important to remember that if the raw data lacks records for a specific hour and area combination, the aggregated data for that period should be marked as 0. This step is crucial for accurate time series prediction.


In [ ]:
-- Creating a table where, for each pair of timestamp/H3, we calculate the number of trips. 
-- We will strip off minutes and seconds and keep only hours

CREATE OR REPLACE TABLE ny_taxi_rides_h3 as
SELECT TIME_SLICE(TO_TIMESTAMP_NTZ(pickup_datetime), 60, 'minute', 'START') AS pickup_datetime,
       H3_POINT_TO_CELL_string(pickup_location, 8) as h3, 
       count(*) as pickups
from ny_taxi_rides
group by 1, 2;

In [ ]:
-- Remove hexagons where, in 2014, there were less than 1M trips.

CREATE OR REPLACE TABLE ny_taxi_rides_h3 AS
WITH all_hexagons AS (
    SELECT h3, 
           SUM(pickups) AS total_pickups
    FROM ny_taxi_rides_h3
    where year(pickup_datetime) = 2014
    GROUP BY 1
)
SELECT t1.*
FROM ny_taxi_rides_h3 t1
INNER JOIN all_hexagons t2 ON t1.h3 = t2.h3
WHERE total_pickups >= 1000000;

In [ ]:
-- For any H3 location and timestamp pair without recorded trips, add records indicating that there were zero trips

CREATE OR REPLACE TABLE ny_taxi_rides_h3 AS
WITH all_dates_hexagons AS (
    SELECT DATEADD(HOUR, VALUE::int, '2014-01-01'::timestamp) AS pickup_datetime, h3
    FROM TABLE(FLATTEN(ARRAY_GENERATE_RANGE(0, DATEDIFF('hour', '2014-01-01', '2015-12-31 23:59:00') + 1)))
    CROSS JOIN (SELECT DISTINCT h3 FROM ny_taxi_rides_h3)
)
SELECT TO_TIMESTAMP_NTZ(t1.pickup_datetime) as pickup_datetime, 
t1.h3, IFF(t2.pickups IS NOT NULL, t2.pickups, 0) AS pickups
FROM all_dates_hexagons t1
LEFT JOIN ny_taxi_rides_h3 t2 ON t1.pickup_datetime = t2.pickup_datetime AND t1.h3 = t2.h3;

## Step 4. Data Enrichment

In this step, we will enhance our dataset with extra features that could improve the accuracy of our predictions. It makes sense to consider that the day of the week and public or school holidays could affect the demand for taxi services. Likewise, areas hosting sporting events might experience a surge in taxi pickups. To incorporate this insight, we will use data from **PredictHQ - Demo**, which provides information on events in New York for the years 2014-2015.

For sports events, we will include only those that have high rank.

In [ ]:
-- Add new features that indicate current dat of the week, whether it was public or school holiday and

create or replace table ny_taxi_rides_h3 as
SELECT t1.*, DAYOFWEEK(t1.pickup_datetime) AS day_of_week,
       IFF(t2.category = 'school-holidays', 'school-holidays', 'None') AS school_holiday,
       IFF(t3.category = 'public-holidays', t3.phq_labels, 'None') AS public_holiday,
       IFF(t4.category = 'sports', t4.labels, 'None') AS sport_event
FROM ny_taxi_rides_h3 t1
LEFT JOIN (select * from PREDICTHQ_DATA where category = 'school-holidays' and title ilike 'New York%') t2 
    ON DATE(t1.pickup_datetime) between t2.start_time AND t2.end_time
LEFT JOIN (select * from PREDICTHQ_DATA where labels ilike '%holiday-national%') t3 
    ON DATE(t1.pickup_datetime) between t3.start_time AND t3.end_time
LEFT JOIN (select * from PREDICTHQ_DATA where rank > 70 and category = 'sports') t4 
    ON t1.pickup_datetime = date_trunc('hour', t4.start_time) 
    AND t1.h3 = h3_point_to_cell_string(t4.geog, 8)

## Step 5. Training and Prediction

In this step, we'll divide our dataset into two parts: the Training set and the Test set. The Training set will be used to train our ML model. It will include data from the entirety of 2014 and part of 2015, going up to the current date. For instance, if today is April 5th, the Training set will cover the period from January 1st, 2014, to April 4th, 2015.

The test set, on the other hand, will contain data for one week starting April 5th, 2015. This setup allows us to evaluate the model's performance on data it hasn't seen during training, providing insight into how well it can predict future taxi trip volumes.

As soon as we have the Training and Test sets, we will run the training for the time series of each hexagon on the Training set, predict the "future" demand using the Test set, and compare it with the actual demand. Given that trip volumes may differ across hexagons, we will use a metric indifferent to absolute values. The Mean Absolute Percentage Error (MAPE) is a viable option.

In [ ]:
-- Create a Training set

create or replace table ny_taxi_rides_h3_train
as select * from ny_taxi_rides_h3
where date(pickup_datetime) < DATEADD(year, -9, current_date())

In [ ]:
-- Create a Test set

CREATE OR REPLACE TABLE ny_taxi_rides_h3_test AS
select h3, 
       pickup_datetime, 
       DAY_OF_WEEK,
       SCHOOL_HOLIDAY,
       PUBLIC_HOLIDAY,
       SPORT_EVENT
from ny_taxi_rides_h3
where date(pickup_datetime) >= DATEADD(year, -9, current_date())
and date(pickup_datetime) < DATEADD(day, 7, DATEADD(year, -9, current_date()))

In [ ]:
-- Train the model using the Training set. We will create a separate model for each H3 cell ID.

create or replace snowflake.ml.forecast my_demand_prediction_model(
  input_data => system$reference('table', 'ny_taxi_rides_h3_train'),
  series_colname => 'h3',
  timestamp_colname => 'pickup_datetime',
  target_colname => 'pickups'
);

In [ ]:
-- Create your predictions for one week of test data and store it in the "forecasts" table.

BEGIN
    CALL my_demand_prediction_model!FORECAST(
        INPUT_DATA => SYSTEM$REFERENCE('TABLE', 'ny_taxi_rides_h3_test'),
        SERIES_COLNAME => 'h3',
        TIMESTAMP_COLNAME => 'pickup_datetime',
        CONFIG_OBJECT => {'prediction_interval': 0.95}
    );
    -- These steps store your predictions to a table.
    LET x := SQLID;
    CREATE or replace TABLE forecasts AS SELECT * FROM TABLE(RESULT_SCAN(:x));
END;

In [ ]:
-- If any forecasts or prediction interval are negative, convert to zero. 
create or replace table forecasts as 
select series::string as h3, 
       ts as pickup_datetime,
       case when forecast < 0 then 0 else forecast end as forecast,
       case when lower_bound < 0 then 0 else lower_bound end as lower_bound,
       case when upper_bound < 0 then 0 else upper_bound end as upper_bound
from forecasts;

In [ ]:
create or replace table prediction_vs_actual as 
select t1.h3, t1.pickup_datetime, t1.pickups, round(t2.forecast) as forecast
from ny_taxi_rides_h3 t1
inner join forecasts t2
on t1.h3 = t2.h3
and t1.pickup_datetime = t2.pickup_datetime

In [ ]:
-- Let's see what is the MAPE value for each of the hexagons

select h3, 
sum(iff(pickups != 0, abs(pickups-forecast)/pickups, abs(pickups-forecast)/(pickups+1)))/168 as mape, 
sum(pickups) as pickups 
from aa_quickstart.public.prediction_vs_actual
group by 1
order by 2 desc;

You can notice that the more trips that occur within a hexagon, the better the prediction. The worst predictions are for hexagons with a relatively low number of trips.

## Step 6. Visualization

In this step we will visualize the actual and predicted results on the map and build a plot showing predicted and actual results for one of the hexagons.